In [5]:
import numpy as np
import pandas as pd
from numpy import random
from numpy.random import randn
import matplotlib.pyplot as plt
from filterpy.kalman import KalmanFilter
import kf_book.book_plots as bp

df = pd.read_csv('sensorData.csv')
uwb_x = df.uwb_x
vel_linear_x = df['vel_linear_x'].fillna(0.0)

In [6]:
len(uwb_x)

1984

In [7]:
def predict(pos, movement):
    return gaussian(pos.mean + movement.mean, pos.var + movement.var)

def gaussian_multiply(g1, g2):
    mean = (g1.var * g2.mean + g2.var * g1.mean) / (g1.var + g2.var)
    variance = (g1.var * g2.var) / (g1.var + g2.var)
    return gaussian(mean, variance)

def update(prior, likelihood):
    posterior = gaussian_multiply(likelihood, prior)
    return posterior

In [8]:
import kf_book.kf_internal as kf_internal
from collections import namedtuple
gaussian = namedtuple('Gaussian', ['mean', 'var'])
gaussian.__repr__ = lambda s: '(={:.3f}, 2={:.3f})'.format(s[0], s[1])
# note: A large variance implies that confidence is very low, 
# so the filter estimates the position to be very close to the measurement
# specific to the robot
process_var = 0.028 ** 2
sensor_var = 0.077 ** 2
# sensor_var = 0.3 ** 2
# initial uwb_x: around 2 when real tag value received and written, otherwise 0.0
x = gaussian(uwb_x[0], sensor_var) # robot's initial position(x-coordinate)

# if changed to 0.1 the kf precision will be lower
dt = 0.01

# measurements are uwb_x
# Kalman filter
print('    PREDICT\t\tMEASURE    UPDATE')
print('  x\t   var\t\tz\t  x\t  var')
# perform kf on measurement
for i, z in enumerate(uwb_x):
    velocity = vel_linear_x[i]
    process_model = gaussian(velocity*dt, process_var)
    prior = predict(x, process_model)
    likelihood = gaussian(z, sensor_var)
    x = update(prior, likelihood)
    
    kf_internal.print_gh(prior, x, z)
print('final estimate X-coordinate:\t{:10.3f} m'.format(x.mean))
print('tag final position:\t\t{:10.3f} m'.format(uwb_x.iloc[-1]))

    PREDICT		MEASURE    UPDATE
  x	   var		z	  x	  var
  0.000    0.007	0.000	  0.000   0.003
  0.000    0.004	0.000	  0.000   0.002
  0.000    0.003	0.000	  0.000   0.002
  0.000    0.003	0.000	  0.000   0.002
  0.000    0.003	0.000	  0.000   0.002
  0.000    0.003	1.880	  0.579   0.002
  0.579    0.003	1.880	  0.977   0.002
  0.977    0.003	1.880	  1.252   0.002
  1.252    0.003	1.880	  1.443   0.002
  1.443    0.003	1.880	  1.576   0.002
  1.576    0.003	1.880	  1.668   0.002
  1.668    0.003	1.880	  1.732   0.002
  1.732    0.003	1.880	  1.777   0.002
  1.777    0.003	1.880	  1.808   0.002
  1.808    0.003	1.870	  1.827   0.002
  1.827    0.003	1.870	  1.840   0.002
  1.840    0.003	1.870	  1.849   0.002
  1.849    0.003	1.870	  1.856   0.002
  1.856    0.003	1.870	  1.860   0.002
  1.860    0.003	1.870	  1.863   0.002
  1.863    0.003	1.870	  1.865   0.002
  1.865    0.003	1.870	  1.867   0.002
  1.867    0.003	1.870	  1.868   0.002
  1.868    0.003	1.890	  1.874   0.002
  1.874  

  2.665    0.003	2.710	  2.678   0.002
  2.683    0.003	2.710	  2.691   0.002
  2.696    0.003	2.710	  2.701   0.002
  2.706    0.003	2.710	  2.707   0.002
  2.712    0.003	2.710	  2.711   0.002
  2.716    0.003	2.710	  2.714   0.002
  2.720    0.003	2.710	  2.717   0.002
  2.722    0.003	2.710	  2.718   0.002
  2.723    0.003	2.710	  2.719   0.002
  2.724    0.003	2.770	  2.738   0.002
  2.743    0.003	2.770	  2.751   0.002
  2.756    0.003	2.770	  2.760   0.002
  2.765    0.003	2.770	  2.767   0.002
  2.772    0.003	2.770	  2.771   0.002
  2.776    0.003	2.770	  2.774   0.002
  2.779    0.003	2.770	  2.777   0.002
  2.782    0.003	2.770	  2.778   0.002
  2.783    0.003	2.770	  2.779   0.002
  2.784    0.003	2.770	  2.780   0.002
  2.785    0.003	2.820	  2.795   0.002
  2.800    0.003	2.820	  2.806   0.002
  2.811    0.003	2.820	  2.814   0.002
  2.819    0.003	2.820	  2.819   0.002
  2.824    0.003	2.820	  2.823   0.002
  2.828    0.003	2.820	  2.826   0.002
  2.831    0.003	2.820	  

  3.858    0.003	3.840	  3.852   0.002
  3.857    0.003	3.800	  3.840   0.002
  3.845    0.003	3.800	  3.831   0.002
  3.836    0.003	3.800	  3.825   0.002
  3.830    0.003	3.800	  3.821   0.002
  3.826    0.003	3.800	  3.818   0.002
  3.823    0.003	3.800	  3.816   0.002
  3.821    0.003	3.800	  3.815   0.002
  3.820    0.003	3.800	  3.814   0.002
  3.819    0.003	3.800	  3.813   0.002
  3.818    0.003	3.800	  3.813   0.002
  3.818    0.003	3.780	  3.806   0.002
  3.811    0.003	3.780	  3.802   0.002
  3.807    0.003	3.780	  3.799   0.002
  3.804    0.003	3.780	  3.797   0.002
  3.802    0.003	3.780	  3.795   0.002
  3.800    0.003	3.780	  3.794   0.002
  3.799    0.003	3.780	  3.793   0.002
  3.799    0.003	3.780	  3.793   0.002
  3.798    0.003	3.780	  3.793   0.002
  3.798    0.003	3.780	  3.792   0.002
  3.798    0.003	3.750	  3.783   0.002
  3.788    0.003	3.750	  3.777   0.002
  3.781    0.003	3.750	  3.772   0.002
  3.777    0.003	3.750	  3.769   0.002
  3.774    0.003	3.750	  

  3.729    0.003	3.710	  3.723   0.002
  3.728    0.003	3.710	  3.723   0.002
  3.727    0.003	3.650	  3.704   0.002
  3.709    0.003	3.650	  3.691   0.002
  3.696    0.003	3.650	  3.682   0.002
  3.687    0.003	3.650	  3.676   0.002
  3.681    0.003	3.650	  3.672   0.002
  3.677    0.003	3.650	  3.669   0.002
  3.674    0.003	3.650	  3.666   0.002
  3.671    0.003	3.650	  3.665   0.002
  3.670    0.003	3.650	  3.664   0.002
  3.669    0.003	3.650	  3.663   0.002
  3.668    0.003	3.600	  3.648   0.002
  3.653    0.003	3.600	  3.637   0.002
  3.642    0.003	3.600	  3.629   0.002
  3.634    0.003	3.600	  3.624   0.002
  3.629    0.003	3.600	  3.620   0.002
  3.625    0.003	3.600	  3.617   0.002
  3.622    0.003	3.600	  3.616   0.002
  3.621    0.003	3.600	  3.614   0.002
  3.619    0.003	3.600	  3.614   0.002
  3.619    0.003	3.600	  3.613   0.002
  3.618    0.003	3.560	  3.600   0.002
  3.605    0.003	3.560	  3.592   0.002
  3.597    0.003	3.560	  3.585   0.002
  3.590    0.003	3.560	  

  2.102    0.003	2.100	  2.102   0.002
  2.102    0.003	2.100	  2.101   0.002
  2.101    0.003	2.100	  2.101   0.002
  2.101    0.003	2.100	  2.101   0.002
  2.101    0.003	2.100	  2.100   0.002
  2.100    0.003	2.100	  2.100   0.002
  2.100    0.003	2.100	  2.100   0.002
  2.100    0.003	2.100	  2.100   0.002
  2.100    0.003	2.100	  2.100   0.002
  2.100    0.003	2.100	  2.100   0.002
  2.100    0.003	2.100	  2.100   0.002
  2.100    0.003	2.100	  2.100   0.002
  2.100    0.003	2.100	  2.100   0.002
  2.100    0.003	2.100	  2.100   0.002
  2.100    0.003	2.100	  2.100   0.002
  2.100    0.003	2.100	  2.100   0.002
  2.100    0.003	2.100	  2.100   0.002
  2.100    0.003	2.110	  2.103   0.002
  2.103    0.003	2.110	  2.105   0.002
  2.105    0.003	2.110	  2.107   0.002
  2.107    0.003	2.110	  2.108   0.002
  2.108    0.003	2.110	  2.108   0.002
  2.108    0.003	2.110	  2.109   0.002
  2.109    0.003	2.110	  2.109   0.002
  2.109    0.003	2.110	  2.109   0.002
  2.109    0.003	2.110	  

In [12]:
from kf_book import book_plots as book_plots
from ipywidgets import interact
from ipywidgets.widgets import IntSlider

# save output in these lists for plotting
xs, predictions = [], []

# process_model = gaussian(velocity, process_var) 

# perform Kalman filter
# x = gaussian(0., 20.**2)
for z in uwb_x:    
    prior = predict(x, process_model)
    likelihood = gaussian(z, sensor_var)
    x = update(prior, likelihood)

    # save results
    predictions.append(prior.mean)
    xs.append(x.mean)

def plot_filter(step):
    plt.cla()
    step -= 1
    i = step // 3 + 1
 
    book_plots.plot_predictions(predictions[:i])    
    zs = uwb_x
    if step % 3 == 0:
        book_plots.plot_measurements(zs[:i-1])
        book_plots.plot_filter(xs[:i-1])
    elif step % 3 == 1:
        book_plots.plot_measurements(zs[:i])
        book_plots.plot_filter(xs[:i-1])
    else:
        book_plots.plot_measurements(zs[:i])
        book_plots.plot_filter(xs[:i])
    
    plt.xlim(-50, len(uwb_x) + 75)
    plt.ylim(min(uwb_x) - 0.5, max(uwb_x) + 0.5)
    plt.xlabel("Number of measurements")
    plt.ylabel("X-coordinate (m)")
#     loc=2
    plt.legend(loc='lower right');
#     plt.savefig('kalman_real_velocity.pdf', dpi=300)
# value denotes the initial value of the slider
interact(plot_filter, step=IntSlider(value=len(predictions)*3, min=1, max=len(predictions)*3));

interactive(children=(IntSlider(value=5952, description='step', max=5952, min=1), Output()), _dom_classes=('wi…